In [1]:
%matplotlib widget

In [2]:
import shutil

from core.gain import *
from core.rnn_predic import *
from core.models import *
from core.util import *
#from core.window import WindowGenerator, MissData, make_dataset_water, WaterDataGenerator
from core.window import WindowGenerator, make_dataset_gain, make_dataset_water
from core.file_open import make_dataframe
from core.miss_data import MissData
import json

import os

In [3]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"


In [4]:
data_path = 'data/'
parameters_dir = 'input'

parameters_file = 'input.json'
parameters_path = '{dir}/{file}'.format(dir=parameters_dir, file=parameters_file)

with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

gain_parameters = parameters['gain']
rnn_parameters = parameters['rnn']
file_parameters = parameters['file']

# input json name

In [5]:
file_parameters['watershed'] = 'yeongsan'

In [6]:
parameters_path = parameters_dir+'/'+ file_parameters['watershed'] + '.json'
with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

In [7]:
pd.set_option('display.max_columns', 1000)

In [8]:
data_parameters = parameters['data']

interpolation_option = data_parameters['interpolation']
colum_idx = data_parameters['columns']
watershed = data_parameters['watershed']
file_names = data_parameters['files']
folder = data_parameters['directorys']
for i in range(len(folder)):
    folder[i] = watershed+folder[i]

In [9]:
__GAIN_TRAINING__ = gain_parameters['train']
gain_epochs = gain_parameters['max_epochs']
gain_in_setps = gain_parameters['input_width']
gain_out_setps = gain_parameters['label_width']
gain_batch_size = gain_parameters['batch_size']
gain_fill_no = gain_parameters['fill_width']
gain_shift = gain_parameters['shift_width']
gain_miss_rate = gain_parameters['miss_rate']

__RNN_TRAINING__ = rnn_parameters['train']
rnn_epochs = rnn_parameters['max_epochs']
rnn_in_setps = rnn_parameters['input_width']
rnn_out_steps = rnn_parameters['label_width']
rnn_batch_size = rnn_parameters['batch_size']
rnn_predict_day = rnn_parameters['predict_day']
rnn_target_column = rnn_parameters['target_column']

if rnn_predict_day < 3 or rnn_predict_day >5:
    print('predict_day err')
    exit(88)
rnn_predict_day -= 1

In [10]:
gain_epochs , rnn_epochs

(2000, 10)

# run file

In [11]:
run_num = range(len(folder))


run_num = [0]




real_df_all = pd.DataFrame([])
target_all = target_mean = target_std = 0

gain_val_performance = {}
gain_performance = {}

length = len(run_num)


# Gain Training Flag

In [12]:
__GAIN_TRAINING__ = False

In [13]:
for i in range(length):

    idx = run_num[i]

    print('interpol flag : ', interpolation_option[idx])
    print('folder : ', data_path + folder[idx])
    print('colum_idx : ', colum_idx[idx])
    print('file_names[idx] : ', file_names[idx])

    #start = time.time()

    #if watershed == '한강_12days_test':
    #    df, times = make_dataframe_temp_12days(folder[idx], file_names[idx], colum_idx[idx], interpolate=interpolation_option[idx])
    #else:
    df, times = make_dataframe(data_path+folder[idx], file_names[idx], colum_idx[idx], interpolation=interpolation_option[idx])

    df_all, train_mean, train_std, df = normalize(df)


    

    if i == 0:
        dfff = df
        target_all = df_all
        target_std = train_std
        target_mean = train_mean
        start_year = str(times.iloc[0].year)
        end_year = str(times.iloc[-1].year)

    if interpolation_option[idx][0] == False:

        loadfiles = ['idx.npy', 'miss.npy', 'discriminator.h5', 'generator.h5']

        gain_calc_falg = True

        if __GAIN_TRAINING__ == True:
            gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
            #print(folder[idx], ': training ', 'Miss date save : ', gain_calc_falg)
        else:
            for file in loadfiles:
                if os.path.isfile('save/' + folder[idx]+file):
                    shutil.copyfile('save/' + folder[idx]+file, 'save/'+file)
                    #print('load file name : save/' + folder[idx]+file)
                else:
                    if file == 'miss.npy':
                        gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
                        #print(folder[idx], ': is not miss.npy ', 'Miss date save : ', gain_calc_falg)

        if gain_calc_falg == True:
            #print('GainWindowGenerator in main')
            WindowGenerator.make_dataset = make_dataset_gain
            wide_window = WindowGenerator(input_width=gain_in_setps, label_width=gain_out_setps, shift=gain_shift,
                                          fill_no=gain_fill_no, miss_rate=gain_miss_rate, batch_size=gain_batch_size,
                                          train_df = df_all, val_df = df_all, test_df = df_all, df = df)

            #gain = model_GAIN(shape=wide_window.dg.shape[1:], gen_sigmoid=False, epochs=gain_epochs, training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')
            gain = model_GAIN(shape=(gain_in_setps, df_all.shape[1]), gen_sigmoid=False, epochs=gain_epochs,
                              training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')

            gain_val_performance[str(i)] = gain.evaluate(wide_window.val)
            gain_performance[str(i)] = gain.evaluate(wide_window.test, verbose=0)

            #print('file proc in main')
            if __GAIN_TRAINING__ == True:
                #dir = 'save/'+folder[i]
                if not os.path.exists('save/' + folder[idx]):
                    os.makedirs('save/'+folder[idx])
                for file in loadfiles:
                    shutil.copyfile('save/' + file, 'save/' + folder[idx] + file)

            #print('create_dataset_with_gain in main')
            #ori, gan = create_dataset_with_gain(gain=gain, window=wide_window, df=df)
            ori, gan = create_dataset_with_gain(gain=gain, shape=(gain_in_setps, df_all.shape[1]), df=df)

        else:
            gan = create_dataset_interpol(window=gain_in_setps, df=df)
    else:
        gan = create_dataset_interpol(window=gain_in_setps, df=df)

    if i == 0 :
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.DataFrame(gan)
    else:
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.concat([real_df_all, pd.DataFrame(gan)], axis=1)

interpol flag :  [False, False]
folder :  data/yeong/자동/
colum_idx :  :,[26,27,28,29,30,31,32,33]
file_names[idx] :  [['나주_2015.xlsx', '나주_2016.xlsx', '나주_2017.xlsx', '나주_2018.xlsx', '나주_2019.xlsx', '나주_2020.xlsx'], ['지석천_2015.xlsx', '지석천_2016.xlsx', '지석천_2017.xlsx', '지석천_2018.xlsx', '지석천_2019.xlsx', '지석천_2020.xlsx'], ['용봉_2015.xlsx', '용봉_2016.xlsx', '용봉_2017.xlsx', '용봉_2018.xlsx', '용봉_2019.xlsx', '용봉_2020.xlsx'], ['우치_2015.xlsx', '우치_2016.xlsx', '우치_2017.xlsx', '우치_2018.xlsx', '우치_2019.xlsx', '우치_2020.xlsx']]
data/yeong/자동/나주_2015.xlsx
data/yeong/자동/나주_2016.xlsx
data/yeong/자동/나주_2017.xlsx
data/yeong/자동/나주_2018.xlsx
data/yeong/자동/나주_2019.xlsx
data/yeong/자동/나주_2020.xlsx
time range in files :  2015-01-01 00:00  ~  2020-12-31 23:00
data/yeong/자동/지석천_2015.xlsx
data/yeong/자동/지석천_2016.xlsx
data/yeong/자동/지석천_2017.xlsx
data/yeong/자동/지석천_2018.xlsx
data/yeong/자동/지석천_2019.xlsx
data/yeong/자동/지석천_2020.xlsx
time range in files :  2015-01-01 00:00  ~  2020-12-31 23:00
data/yeong/자동/용봉_2015.xlsx
data/

In [14]:
print(real_df_all.shape)

(52608, 48)


In [83]:
def dataset_slice(df, train_ratio, val_ratio, test_ratio):
    total_no = df.shape[0]

    train_no = int(total_no * train_ratio)
    #
    val_no = int(total_no * (train_ratio + val_ratio))

    # val_no = int(total_no*val_ratio)
    # train_no =int(total_no * (train_ratio+val_ratio))
    #
    # val_slice = slice(0, val_no)
    # train_slice = slice(val_no, train_no)
    # test_slice = slice(train_no, None)

    train_slice = slice(0, train_no)    #0.8
    val_slice = slice(train_no, val_no)   #0.1
    test_slice = slice(val_no, None)
    # val_slice = slice(train_no, val_no)   #0.1
    # test_slice = slice(val_no, None)
#     train_slice = slice(0, train_no)  # 0.8
#     val_slice = slice(train_no, None)  # 0.1
#     test_slice = slice(val_no, None)

    train = pd.DataFrame(df[train_slice])
    val = pd.DataFrame(df[val_slice])
    test = pd.DataFrame(df[test_slice])

    test_slice2 = slice(total_no - 288, None)
    test2 = pd.DataFrame(df[test_slice2])

    #print('total_no :1111111111111111111111111 ')
    #print('total_no : ', total_no)
    #print('train : ', train.shape)
    #print('val : ', val.shape)
    #print('test : ', test.shape)


    return train, val, test, test2

# slice DF

In [128]:
fake_df=real_df_all.iloc[:-2000, :]

In [129]:
test_df, val_df, train_df, test_df2 = dataset_slice(fake_df, 0.8, 0.1, 0.1)

In [130]:
val_df.shape, test_df.shape, train_df.shape

((5061, 48), (40486, 48), (5061, 48))

In [131]:
print('-------------------prediction')
print('-------------------prediction')
print('-------------------prediction')

print('real_df_all.type : ', type(real_df_all))
print('train_df.type : ', type(train_df))
print('train_df.shape : ', train_df.shape, 'val_df.shape : ', val_df.shape, 'test_df.shape:' ,test_df.shape)


-------------------prediction
-------------------prediction
-------------------prediction
real_df_all.type :  <class 'pandas.core.frame.DataFrame'>
train_df.type :  <class 'pandas.core.frame.DataFrame'>
train_df.shape :  (5061, 48) val_df.shape :  (5061, 48) test_df.shape: (40486, 48)


# target column

In [132]:

label_columns_indices = {name: i for i, name in enumerate(dfff[0])}

print("label_columns_indices:")
print(label_columns_indices)


target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}

rnn_target_column = "tn"

print('target columns : ', rnn_target_column)
num_features = dfff[0].shape[1]



target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
out_features = [target_col_idx]
out_num_features = len(out_features)

print("target_col_idx : ", target_col_idx)
print('out_num_features : ', out_num_features)


label_columns_indices:
{'tmpr_value': 0, 'ph_value': 1, 'do_value': 2, 'ec_value': 3, 'toc_value': 4, '총질소_값': 5, '총인_값': 6, '클로로필-a_값': 7, 'Day sin': 8, 'Day cos': 9, 'Year sin': 10, 'Year cos': 11}
target columns :  tn
target_col_idx :  5
out_num_features :  1


In [133]:
val_nse = {}
val_pbias = {}


WindowGenerator.make_dataset = make_dataset_water
multi_window = WindowGenerator(
    input_width=rnn_in_setps,label_width=rnn_out_steps, shift=rnn_out_steps,out_features=out_features,
    out_num_features=out_num_features,label_columns=dfff[0].columns, batch_size=rnn_batch_size,
    train_df=train_df, val_df=val_df, test_df=test_df, test_df2=test_df2)

if __RNN_TRAINING__:
    if not os.path.exists('save/' + watershed):
        os.makedirs('save/' + watershed)


idx = [2, 4, 5, 6, 7]
pa = ["do/", "toc/", "nitrogen/", "phosphorus/", "chlorophyll-a/"]

indices = {name: i for i, name in enumerate(idx)}

model_path = "save/" + watershed + "models/" + pa[indices[target_col_idx]]
print("save model path : ", model_path)

val_nse = {}
val_pbias = {}

 # +"gru.ckpt" -- path




save model path :  save/yeong/models/nitrogen/


In [134]:
plt.figure()
plt.plot(real_df_all.iloc[:,out_features[0]])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [135]:
plt.figure()
plt.plot(test_df.iloc[:,out_features[0]])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## 모델 학습

In [136]:
__RNN_TRAINING__ = True

In [137]:
rnn_epochs = 200
# multi_linear_model = model_multi_linear(
#     window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
#     #training_flag=__RNN_TRAINING__, checkpoint_path="save/"+watershed+"models/multi_linear.ckpt")
#     training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"multi_linear.ckpt")
# elman_model = model_elman(
#     window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
#     training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"elman.ckpt")
gru_model = model_gru(
    window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
    training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"gru.ckpt")
multi_lstm_model = model_multi_lstm(
    window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
    training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"multi_lstm.ckpt")
multi_conv_model = model_multi_conv(
    window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
    training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"multi_conv.ckpt")


Epoch 1/200
1/1 [==============================] - 3s 3s/step - loss: 0.3339 - mean_absolute_error: 0.4620 - nse: -1.3971e-04 - val_loss: 1.0129 - val_mean_absolute_error: 0.7924 - val_nse: -0.4861

Epoch 00001: val_loss improved from inf to 1.01293, saving model to save/yeong/models/nitrogen/gru.ckpt
Epoch 2/200
1/1 [==============================] - 1s 529ms/step - loss: 0.2918 - mean_absolute_error: 0.4423 - nse: 0.0402 - val_loss: 0.9640 - val_mean_absolute_error: 0.7598 - val_nse: -0.4365

Epoch 00002: val_loss improved from 1.01293 to 0.96398, saving model to save/yeong/models/nitrogen/gru.ckpt
Epoch 3/200
1/1 [==============================] - 1s 516ms/step - loss: 0.2753 - mean_absolute_error: 0.4239 - nse: 0.1723 - val_loss: 0.7177 - val_mean_absolute_error: 0.6433 - val_nse: -0.1652

Epoch 00003: val_loss improved from 0.96398 to 0.71766, saving model to save/yeong/models/nitrogen/gru.ckpt
Epoch 4/200
1/1 [==============================] - 1s 571ms/step - loss: 0.2231 - mean_

1/1 [==============================] - 1s 567ms/step - loss: 0.0863 - mean_absolute_error: 0.2122 - nse: 0.7264 - val_loss: 0.5662 - val_mean_absolute_error: 0.5940 - val_nse: 0.1587

Epoch 00030: val_loss did not improve from 0.42137
Epoch 31/200
1/1 [==============================] - 1s 562ms/step - loss: 0.0637 - mean_absolute_error: 0.1839 - nse: 0.8107 - val_loss: 0.4343 - val_mean_absolute_error: 0.5080 - val_nse: 0.3372

Epoch 00031: val_loss did not improve from 0.42137
Epoch 32/200
1/1 [==============================] - 1s 545ms/step - loss: 0.0692 - mean_absolute_error: 0.1944 - nse: 0.7567 - val_loss: 0.4273 - val_mean_absolute_error: 0.5044 - val_nse: 0.3505

Epoch 00032: val_loss did not improve from 0.42137
Epoch 33/200
1/1 [==============================] - 1s 541ms/step - loss: 0.0635 - mean_absolute_error: 0.1869 - nse: 0.8051 - val_loss: 0.5221 - val_mean_absolute_error: 0.5844 - val_nse: 0.2297

Epoch 00033: val_loss did not improve from 0.42137
Epoch 34/200
1/1 [===

1/1 [==============================] - 1s 531ms/step - loss: 0.0507 - mean_absolute_error: 0.1699 - nse: 0.8175 - val_loss: 0.3819 - val_mean_absolute_error: 0.4959 - val_nse: 0.4271

Epoch 00062: val_loss did not improve from 0.31124
Epoch 63/200
1/1 [==============================] - 1s 559ms/step - loss: 0.0543 - mean_absolute_error: 0.1715 - nse: 0.8274 - val_loss: 0.3838 - val_mean_absolute_error: 0.5042 - val_nse: 0.3468

Epoch 00063: val_loss did not improve from 0.31124
Epoch 64/200
1/1 [==============================] - 1s 568ms/step - loss: 0.0532 - mean_absolute_error: 0.1705 - nse: 0.8195 - val_loss: 0.3581 - val_mean_absolute_error: 0.4628 - val_nse: 0.5510

Epoch 00064: val_loss did not improve from 0.31124
Epoch 65/200
1/1 [==============================] - 1s 533ms/step - loss: 0.0522 - mean_absolute_error: 0.1682 - nse: 0.7986 - val_loss: 0.3141 - val_mean_absolute_error: 0.4393 - val_nse: 0.5225

Epoch 00065: val_loss did not improve from 0.31124
Epoch 66/200
1/1 [===

1/1 [==============================] - 1s 528ms/step - loss: 0.0437 - mean_absolute_error: 0.1557 - nse: 0.8743 - val_loss: 0.4779 - val_mean_absolute_error: 0.5357 - val_nse: 0.2565

Epoch 00095: val_loss did not improve from 0.31124
Epoch 96/200
1/1 [==============================] - 1s 533ms/step - loss: 0.0475 - mean_absolute_error: 0.1619 - nse: 0.8510 - val_loss: 0.4124 - val_mean_absolute_error: 0.4917 - val_nse: 0.3217

Epoch 00096: val_loss did not improve from 0.31124
Epoch 97/200
1/1 [==============================] - 1s 506ms/step - loss: 0.0450 - mean_absolute_error: 0.1528 - nse: 0.7914 - val_loss: 0.5285 - val_mean_absolute_error: 0.5807 - val_nse: 0.2239

Epoch 00097: val_loss did not improve from 0.31124
Epoch 98/200
1/1 [==============================] - 1s 540ms/step - loss: 0.0421 - mean_absolute_error: 0.1533 - nse: 0.8335 - val_loss: 0.4879 - val_mean_absolute_error: 0.5594 - val_nse: 0.3128

Epoch 00098: val_loss did not improve from 0.31124
Epoch 99/200
1/1 [===

Epoch 128/200
1/1 [==============================] - 1s 525ms/step - loss: 0.0434 - mean_absolute_error: 0.1501 - nse: 0.8504 - val_loss: 0.4603 - val_mean_absolute_error: 0.5316 - val_nse: 0.4159

Epoch 00128: val_loss did not improve from 0.31124
Epoch 129/200
1/1 [==============================] - 1s 549ms/step - loss: 0.0385 - mean_absolute_error: 0.1460 - nse: 0.8497 - val_loss: 0.6276 - val_mean_absolute_error: 0.6286 - val_nse: 0.0209

Epoch 00129: val_loss did not improve from 0.31124
Epoch 130/200
1/1 [==============================] - 1s 576ms/step - loss: 0.0423 - mean_absolute_error: 0.1514 - nse: 0.8597 - val_loss: 0.4980 - val_mean_absolute_error: 0.5492 - val_nse: 0.2778

Epoch 00130: val_loss did not improve from 0.31124
Epoch 131/200
1/1 [==============================] - 1s 548ms/step - loss: 0.0382 - mean_absolute_error: 0.1444 - nse: 0.8760 - val_loss: 0.4694 - val_mean_absolute_error: 0.5437 - val_nse: 0.1113

Epoch 00131: val_loss did not improve from 0.31124
Epoc

...


Epoch 00053: val_loss did not improve from 0.33070
Epoch 54/200
1/1 [==============================] - 0s 264ms/step - loss: 0.0346 - mean_absolute_error: 0.1397 - nse: 0.8788 - val_loss: 0.5684 - val_mean_absolute_error: 0.5838 - val_nse: 0.1955

Epoch 00054: val_loss did not improve from 0.33070
Epoch 55/200
1/1 [==============================] - 0s 284ms/step - loss: 0.0394 - mean_absolute_error: 0.1473 - nse: 0.8811 - val_loss: 0.5093 - val_mean_absolute_error: 0.5558 - val_nse: 0.2280

Epoch 00055: val_loss did not improve from 0.33070
Epoch 56/200
1/1 [==============================] - 0s 282ms/step - loss: 0.0440 - mean_absolute_error: 0.1542 - nse: 0.8598 - val_loss: 0.4776 - val_mean_absolute_error: 0.5434 - val_nse: 0.2743

Epoch 00056: val_loss did not improve from 0.33070
Epoch 57/200
1/1 [==============================] - 0s 263ms/step - loss: 0.0369 - mean_absolute_error: 0.1425 - nse: 0.8539 - val_loss: 0.4827 - val_mean_absolute_error: 0.5250 - val_nse: 0.2252

Epoch 0

1/1 [==============================] - 0s 240ms/step - loss: 0.0375 - mean_absolute_error: 0.1400 - nse: 0.8722 - val_loss: 0.7063 - val_mean_absolute_error: 0.6309 - val_nse: -0.0807

Epoch 00087: val_loss did not improve from 0.33070
Epoch 88/200
1/1 [==============================] - 0s 268ms/step - loss: 0.0311 - mean_absolute_error: 0.1295 - nse: 0.8883 - val_loss: 0.5581 - val_mean_absolute_error: 0.5722 - val_nse: 0.1236

Epoch 00088: val_loss did not improve from 0.33070
Epoch 89/200
1/1 [==============================] - 0s 254ms/step - loss: 0.0342 - mean_absolute_error: 0.1361 - nse: 0.9057 - val_loss: 0.6439 - val_mean_absolute_error: 0.6234 - val_nse: -0.0451

Epoch 00089: val_loss did not improve from 0.33070
Epoch 90/200
1/1 [==============================] - 0s 296ms/step - loss: 0.0284 - mean_absolute_error: 0.1256 - nse: 0.9083 - val_loss: 0.6577 - val_mean_absolute_error: 0.6349 - val_nse: 0.0214

Epoch 00090: val_loss did not improve from 0.33070
Epoch 91/200
1/1 [=

1/1 [==============================] - 0s 235ms/step - loss: 0.0285 - mean_absolute_error: 0.1246 - nse: 0.9014 - val_loss: 0.7545 - val_mean_absolute_error: 0.6797 - val_nse: -0.1744

Epoch 00120: val_loss did not improve from 0.33070
Epoch 121/200
1/1 [==============================] - 0s 276ms/step - loss: 0.0289 - mean_absolute_error: 0.1254 - nse: 0.9048 - val_loss: 0.7442 - val_mean_absolute_error: 0.6550 - val_nse: -0.0245

Epoch 00121: val_loss did not improve from 0.33070
Epoch 122/200
1/1 [==============================] - 0s 290ms/step - loss: 0.0291 - mean_absolute_error: 0.1234 - nse: 0.8877 - val_loss: 0.8354 - val_mean_absolute_error: 0.6883 - val_nse: -0.1375

Epoch 00122: val_loss did not improve from 0.33070
Epoch 123/200
1/1 [==============================] - 0s 300ms/step - loss: 0.0261 - mean_absolute_error: 0.1200 - nse: 0.9309 - val_loss: 0.8035 - val_mean_absolute_error: 0.6715 - val_nse: -0.1182

Epoch 00123: val_loss did not improve from 0.33070
Epoch 124/200


1/1 [==============================] - 0s 272ms/step - loss: 0.0222 - mean_absolute_error: 0.1140 - nse: 0.9311 - val_loss: 0.6990 - val_mean_absolute_error: 0.6167 - val_nse: -0.0736

Epoch 00153: val_loss did not improve from 0.33070
Epoch 154/200
1/1 [==============================] - 0s 272ms/step - loss: 0.0233 - mean_absolute_error: 0.1155 - nse: 0.9169 - val_loss: 0.7632 - val_mean_absolute_error: 0.6669 - val_nse: -0.0799

Epoch 00154: val_loss did not improve from 0.33070
Epoch 155/200
1/1 [==============================] - 0s 265ms/step - loss: 0.0286 - mean_absolute_error: 0.1231 - nse: 0.9155 - val_loss: 0.8093 - val_mean_absolute_error: 0.6779 - val_nse: -0.2042

Epoch 00155: val_loss did not improve from 0.33070
Epoch 156/200
1/1 [==============================] - 0s 260ms/step - loss: 0.0253 - mean_absolute_error: 0.1185 - nse: 0.9178 - val_loss: 0.6654 - val_mean_absolute_error: 0.6138 - val_nse: -0.0527

Epoch 00156: val_loss did not improve from 0.33070
Epoch 157/200


1/1 [==============================] - 0s 289ms/step - loss: 0.0265 - mean_absolute_error: 0.1188 - nse: 0.9065 - val_loss: 0.9199 - val_mean_absolute_error: 0.7181 - val_nse: -0.2191

Epoch 00186: val_loss did not improve from 0.33070
Epoch 187/200
1/1 [==============================] - 0s 255ms/step - loss: 0.0243 - mean_absolute_error: 0.1138 - nse: 0.9341 - val_loss: 1.0111 - val_mean_absolute_error: 0.7600 - val_nse: -0.4793

Epoch 00187: val_loss did not improve from 0.33070
Epoch 188/200
1/1 [==============================] - 0s 253ms/step - loss: 0.0251 - mean_absolute_error: 0.1199 - nse: 0.9309 - val_loss: 1.0239 - val_mean_absolute_error: 0.7749 - val_nse: -0.5611

Epoch 00188: val_loss did not improve from 0.33070
Epoch 189/200
1/1 [==============================] - 0s 234ms/step - loss: 0.0269 - mean_absolute_error: 0.1171 - nse: 0.8957 - val_loss: 1.0257 - val_mean_absolute_error: 0.7877 - val_nse: -0.5784

Epoch 00189: val_loss did not improve from 0.33070
Epoch 190/200


## core / window.py / 

In [124]:
def hour_to_day_mean(array):
    time = 24
    array = array.reshape((array.shape[0], array.shape[1] // time, time, array.shape[2]))
    array = array.mean(2)
    return array

def compa(model=None,df = None, plot_col=0, input_width=7*24, label_width=5*24, target_std=None, target_mean=None, predict_day=4):
    
    print(df.shape)
    print(plot_col)
    
    width = input_width + label_width
    
    length = df.shape[0]
    length -= width
    
    inputs = []
    labels = []
   
    for i in range(0,length,24):
#         print('i = ', i)
        dataset = df.iloc[i:i+width].to_numpy()
        input = dataset[:input_width]
        label = dataset[input_width:, plot_col:plot_col+1]
        
        input = input.reshape((-1,)+input.shape)
        label = label.reshape((-1,)+label.shape)
        
        inputs.append(input)
        labels.append(label)
        
    inputs = np.concatenate(inputs, axis=0)
    labels = np.concatenate(labels, axis=0)
    
#     print('labels.mean(axis=1)')
#     print(labels.mean(axis=1).shape)
    print(inputs.shape)
    print(labels.shape)

    predictions = model(inputs)
    print(predictions.shape)
    
    predictions = predictions.numpy() * target_std[plot_col] + target_mean[plot_col]
    labels = labels * target_std[plot_col] + target_mean[plot_col]

    print('predictions.shape =', predictions.shape)
#     pred_day = predictions
    pred_day = hour_to_day_mean(predictions)
    print('pred_day.shape =', pred_day.shape)
#     label_day= labels
    label_day = hour_to_day_mean(labels)
    
    inputs_target = inputs[:,:,plot_col:plot_col+1]
    inputs_target = inputs_target * target_std[plot_col] + target_mean[plot_col]
#     inputs_day = inputs_target
    inputs_day = hour_to_day_mean(inputs_target)
    
    #plt.figure(figsize=(10, 800))
    
#     input_index = np.array(range(0,length,24))
#     label_index = input_index + 24*7
    
#     print(input_index)
#     print(label_index)
    
    plt.figure()
    plt.plot(label_day[:, predict_day, :], label='label')
    plt.plot(pred_day[:, predict_day, :], label='pred')
#     plt.plot(input_index, inputs_day[:, 0, :], label='input')
#     plt.plot(label_index, label_day[:, 0, :], label='label')
#     plt.plot(label_index, pred_day[:, 0, :], label='pred')
    plt.legend()
        
        
        
#     o1 = np.mean(labels)
# #     print('labels =', labels)
#     print('o1 =', o1)
#     nse1 = ((labels - predictions)**2).sum()
#     nse2 = ((labels - o1)**2).sum()
#     nse3 = 1 - (nse1/nse2)
    
    
    
#     print('pred_day.shape =',pred_day.shape)
#     print('label_day.shape =',label_day.shape)
    #print('label_index =', label_index)
    
    o1 = np.mean(label_day[:,predict_day,0])
#     o1 = np.mean(label_day)
#     print('labels =', labels)
#     o1 = target_mean[plot_col]
    print('o1 =', o1)
    nse1 = ((pred_day-label_day)**2).sum(axis=0)
    nse2 = ((label_day - o1)**2).sum(axis=0)
    nse3 = 1 - (nse1[predict_day]/nse2[predict_day])
    print('nse3 =', nse3)
    #mean = np.mean(label_day[predict_day])
    print('label_day[predict_day].shape =', label_day[predict_day].shape)
#     print(type(pred_day))
#     print(pred_day[:20, 0, 0])
#     print(label_day[:20, 0, 0])
    o = label_day[:, 0, 0]
    s = pred_day[:, 0, 0]
    mean = np.mean(o)
    print('mean =', mean)
    nse = 1. - np.sum((o-s)**2)/np.sum((o-mean)**2)
#     print('nse =', nse)
    
    
#     pbias1 = (label_day - pred_day).sum(axis=0)
#     pbias2 = (label_day).sum(axis=0)
#     pbias3 = (pbias1[predict_day]/pbias2[predict_day])*100
    
    
#     o1 = np.mean(label_day)
# #     print('labels =', labels)
#     print('o1 =', o1)
#     nse1 = ((label_day - pred_day)**2).sum()
#     nse2 = ((label_day - o1)**2).sum()
#     nse3 = 1 - (nse1/nse2)
#     print('nse3 =', nse3)
    #mean = np.mean(label_day[predict_day])
#     print(type(pred_day))
#     print(pred_day[:20, 0, 0])
#     print(label_day[:20, 0, 0])
#     o = label_day[:, 0, 0]
#     s = pred_day[:, 0, 0]
#     mean = np.mean(o)
#     print('mean =', mean)
#     nse = 1. - np.sum((o-s)**2)/np.sum((o-mean)**2)
#     print('nse =', nse)
    
    
    pbias1 = (label_day - pred_day).sum(axis=0)
    pbias2 = (label_day).sum(axis=0)
    pbias3 = (pbias1[predict_day]/pbias2[predict_day])*100
    
    
    
    
    return nse3, np.abs(pbias3), pred_day, labels
    

In [120]:
test_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47
0,-1.506584,-0.781585,0.609397,1.684159,-0.388460,0.914476,-0.137610,-0.525071,1.747686e-12,1.414210,0.002127,1.413681,-1.689783,-0.004983,0.994299,-0.463754,-1.254349,0.019940,-1.432135,-0.901498,1.747686e-12,1.414210,0.002127,1.413681,-1.356208,-0.588429,0.154454,1.470927,-1.231157,1.157017,-0.740015,-0.865323,1.747686e-12,1.414210,0.002127,1.413681,-1.507453,0.396912,0.801488,-0.042523,-1.445851,-0.430887,-0.021002,-0.837544,1.747686e-12,1.414210,0.002127,1.413681
1,-1.510089,-0.797194,0.583524,1.684159,-0.339509,0.940824,-0.116095,-0.537748,3.660245e-01,1.366022,0.003141,1.413679,-1.703138,-0.056581,0.980826,-0.440851,-1.252129,0.066557,-1.446334,-0.901628,3.660245e-01,1.366022,0.003141,1.413679,-1.367382,-0.588860,0.147153,1.470137,-1.172248,1.150839,-0.761529,-0.863685,3.660245e-01,1.366022,0.003141,1.413679,-1.530273,0.212962,0.685987,-0.009650,-1.419883,-0.440716,-0.030038,-0.838586,3.660245e-01,1.366022,0.003141,1.413679
2,-1.514180,-0.801723,0.557911,1.679321,-0.397348,0.916532,-0.116095,-0.548444,7.071051e-01,1.224742,0.004155,1.413677,-1.721294,-0.082758,0.978333,-0.447071,-1.250806,0.057947,-1.503778,-0.900590,7.071051e-01,1.224742,0.004155,1.413677,-1.375527,-0.588900,0.148847,1.459278,-1.273414,1.088776,-0.818973,-0.870471,7.071051e-01,1.224742,0.004155,1.413677,-1.551150,0.065689,0.593695,-0.020410,-1.389266,-0.424052,-0.042516,-0.839976,7.071051e-01,1.224742,0.004155,1.413677
3,-1.515415,-0.815917,0.552050,1.673497,-0.220514,0.915128,-0.116095,-0.551085,9.999976e-01,0.999998,0.005168,1.413674,-1.736147,-0.104176,0.983347,-0.488828,-1.255655,0.106997,-1.514535,-0.899293,9.999976e-01,0.999998,0.005168,1.413674,-1.381628,-0.588456,0.155937,1.467275,-1.280091,1.089618,-0.847587,-0.868599,9.999976e-01,0.999998,0.005168,1.413674,-1.573655,-0.036403,0.538223,-0.068585,-1.483433,-0.405517,-0.051552,-0.839976,9.999976e-01,0.999998,0.005168,1.413674
4,-1.518370,-0.796628,0.560990,1.676459,-0.406037,0.915223,-0.094581,-0.542766,1.224742e+00,0.707105,0.006182,1.413669,-1.745111,-0.117089,0.990270,-0.501366,-1.248377,0.123470,-1.424820,-0.899163,1.224742e+00,0.707105,0.006182,1.413669,-1.389343,-0.588900,0.160290,1.494817,-1.296875,1.167127,-0.847587,-0.866727,1.224742e+00,0.707105,0.006182,1.413669,-1.589207,-0.100847,0.504041,-0.044103,-1.431635,-0.403551,-0.064030,-0.839281,1.224742e+00,0.707105,0.006182,1.413669
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41281,1.173564,1.327393,-0.175488,-0.919043,0.143182,0.091736,-0.653957,0.093643,3.660245e-01,1.366022,-1.368961,-0.355847,1.013118,0.923018,-0.592118,-1.107595,0.671316,-0.860180,0.572367,0.280485,3.660245e-01,1.366022,-1.368961,-0.355847,1.177576,-0.649557,0.238414,0.257186,0.167853,0.694025,-0.973016,1.827672,3.660245e-01,1.366022,-1.368961,-0.355847,0.881833,-0.154268,-0.190047,-0.883157,-0.002714,-1.056228,-0.279055,-0.325991,3.660245e-01,1.366022,-1.368961,-0.355847
41282,1.172562,1.259997,-0.172312,-0.915094,0.106184,0.109243,-0.691607,0.066306,7.071051e-01,1.224742,-1.369215,-0.354866,0.998077,0.765756,-0.626175,-1.111544,0.585716,-0.839960,0.545474,0.013794,7.071051e-01,1.224742,-1.369215,-0.354866,1.161531,-0.806818,0.076758,0.262615,0.156240,0.763295,-0.955159,1.696780,7.071051e-01,1.224742,-1.369215,-0.354866,0.787135,-0.068091,-0.236341,-0.809628,-0.075252,-1.137870,-0.152700,-0.374598,7.071051e-01,1.224742,-1.369215,-0.354866
41283,1.156518,1.147661,-0.161413,-0.916081,0.141006,-0.040720,-0.711400,0.051942,9.999976e-01,0.999998,-1.369469,-0.353885,0.984037,0.586038,-0.661820,-1.099697,0.460931,-0.839960,0.471895,0.052215,9.999976e-01,0.999998,-1.369469,-0.353885,1.146489,-0.851744,-0.033131,0.261628,0.172929,0.774623,-0.951502,1.647435,9.999976e-01,0.999998,-1.369469,-0.35

In [125]:
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(model=gru_model,
                                                      df=test_df,
                                                      plot_col=out_features[0], 
                                                      target_std=target_std, 
                                                      target_mean=target_mean,
                                                      predict_day = 4)

(val_nse['GRU'])
print(val_pbias['GRU'])

(41286, 48)
5
(1709, 168, 48)
(1709, 120, 1)
(1709, 120, 1)
predictions.shape = (1709, 120, 1)
pred_day.shape = (1709, 5, 1)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

o1 = 4.264279
nse3 = [0.3554865]
label_day[predict_day].shape = (5, 1)
mean = 4.266996
[12.418887]


In [126]:
val_nse['LSTM'], val_pbias['LSTM'], pred, label = compa(model=multi_lstm_model,
                                                      df=test_df,
                                                      plot_col=out_features[0], 
                                                      target_std=target_std, 
                                                      target_mean=target_mean,
                                                      predict_day = 4)

(val_nse['LSTM'])
print(val_pbias['LSTM'])

(41286, 48)
5
(1709, 168, 48)
(1709, 120, 1)
(1709, 120, 1)
predictions.shape = (1709, 120, 1)
pred_day.shape = (1709, 5, 1)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

o1 = 4.264279
nse3 = [0.13001621]
label_day[predict_day].shape = (5, 1)
mean = 4.266996
[15.465405]


In [127]:
val_nse['CNN'], val_pbias['CNN'], pred, label = compa(model=multi_conv_model,
                                                      df=test_df,
                                                      plot_col=out_features[0], 
                                                      target_std=target_std, 
                                                      target_mean=target_mean,
                                                      predict_day = 4)

(val_nse['CNN'])
print(val_pbias['CNN'])

(41286, 48)
5
(1709, 168, 48)
(1709, 120, 1)
(1709, 120, 1)
predictions.shape = (1709, 120, 1)
pred_day.shape = (1709, 5, 1)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

o1 = 4.264279
nse3 = [0.41306144]
label_day[predict_day].shape = (5, 1)
mean = 4.266996
[10.212532]


In [58]:
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    model=gru_model,df=train_df,
    plot_col=out_features[0], target_std=target_std, target_mean=target_mean, predict_day = 4)

print()
print()

print(val_nse['GRU'])
print(val_pbias['GRU'])

(42086, 48)
4
(1742, 168, 48)
(1742, 120, 1)
(1742, 120, 1)
predictions.shape = (1742, 120, 1)
pred_day.shape = (1742, 5, 1)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

o1 = 4.2740355
nse3 = [0.70065904]
label_day[predict_day].shape = (5, 1)
mean = 4.2766314


[0.70065904]
[0.6245054]


In [59]:
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    model=gru_model,df=real_df_all,
    plot_col=out_features[0], target_std=target_std, target_mean=target_mean, predict_day = 4)

print()
print()

print(val_nse['GRU'])
print(val_pbias['GRU'])

(52608, 48)
4
(2180, 168, 48)
(2180, 120, 1)
(2180, 120, 1)
predictions.shape = (2180, 120, 1)
pred_day.shape = (2180, 5, 1)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

o1 = 4.27052
nse3 = [0.6760868]
label_day[predict_day].shape = (5, 1)
mean = 4.2712


[0.6760868]
[0.57269627]
